In [19]:
import numpy as np
import pandas as pd
import os
from scipy.signal import welch

In [20]:

# 🔹 File Paths
eeg_data_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/chbmit_preprocessed_data.csv"
seizure_info_path = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/sizuretimes.xlsx"

# 🔹 Output Folder (Create if not exists)
output_folder = "/Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final"
os.makedirs(output_folder, exist_ok=True)

# 🔹 Load Patient List from Seizure Info Excel (Extract Sheet Names)
xls = pd.ExcelFile(seizure_info_path)
patient_ids = xls.sheet_names  # Extract patient IDs dynamically

In [21]:
# 🔹 EEG Sampling Rate
fs = 256  # Hz

# 🔹 Load Preprocessed EEG Data
def load_eeg_data():
    if not os.path.exists(eeg_data_path):
        print(f"❌ File not found: {eeg_data_path}")
        return None
    
    df = pd.read_csv(eeg_data_path)
    print("📂 EEG Data Loaded. Shape:", df.shape)
    return df

In [22]:
# 🔹 Function to Compute Dominant Frequency
def compute_dominant_frequency(eeg_signals):
    dominant_freq_results = {}

    for channel in eeg_signals.columns:
        if eeg_signals[channel].isna().all():  # Skip empty channels
            continue
        
        # Compute Power Spectral Density (PSD) using Welch’s method
        f, Pxx = welch(eeg_signals[channel].dropna(), fs=fs, nperseg=2*fs, window='hann', scaling='density')

        # ✅ Find the frequency with the highest power (dominant frequency) within 1-50 Hz
        valid_range = (f >= 1) & (f <= 50)
        if np.any(valid_range):
            dominant_freq = f[valid_range][np.argmax(Pxx[valid_range])]
        else:
            dominant_freq = np.nan  # If no valid frequencies found

        dominant_freq_results[channel] = dominant_freq  # ✅ Store dominant frequency per electrode
    
    # Convert to DataFrame
    dominant_freq_df = pd.DataFrame.from_dict(dominant_freq_results, orient='index', columns=["Dominant Frequency (Hz)"])
    return dominant_freq_df

In [24]:
# 🔹 Process Each Patient
df = load_eeg_data()
if df is not None:
    for patient in patient_ids:
        print(f"\n📂 Processing {patient}...")

        # ✅ Load seizure data for this patient
        seizure_df = pd.read_excel(seizure_info_path, sheet_name=patient, engine="openpyxl")

        # ✅ Fix column name issue (remove trailing spaces)
        seizure_df.columns = seizure_df.columns.str.strip()

        # ✅ Check if `start` and `stop` columns exist
        if "start" not in seizure_df.columns or "stop" not in seizure_df.columns:
            print(f"⚠️ Skipping {patient}: 'start' or 'stop' columns not found.")
            continue  # Skip if no valid columns

        # ✅ Extract seizure time rows
        patient_rows = seizure_df[['start', 'stop']].dropna()

        if patient_rows.empty:
            print(f"⚠️ Skipping {patient}: No valid seizure times found.")
            continue

        # ✅ Select EEG data for the given seizure times
        seizure_indices = []
        for _, row in patient_rows.iterrows():
            seizure_indices.extend(range(int(row["start"]), int(row["stop"]) + 1))
        
        seizure_indices = [idx for idx in seizure_indices if idx < len(df)]  # Ensure index is within bounds
        patient_eeg_data = df.iloc[seizure_indices, :-1]  # Select relevant EEG channels

        if patient_eeg_data.empty:
            print(f"⚠️ Skipping {patient}: No EEG data found for seizure periods.")
            continue

        # ✅ Compute Dominant Frequency for this patient
        dominant_freq_df = compute_dominant_frequency(patient_eeg_data)

        if dominant_freq_df.empty:
            print(f"⚠️ Skipping {patient}: No valid dominant frequency data found.")
            continue

        # 🔹 Save each patient's dominant frequency as a separate CSV file
        output_file = os.path.join(output_folder, f"{patient}_dominant_frequency.csv")
        dominant_freq_df.to_csv(output_file)
        
        print(f"✅ Saved: {output_file}")

print("\n✅ Dominant Frequency Computation Completed for All Patients!")


📂 EEG Data Loaded. Shape: (2097150, 24)

📂 Processing chb24...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb24_dominant_frequency.csv

📂 Processing chb23...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb23_dominant_frequency.csv

📂 Processing chb22...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb22_dominant_frequency.csv

📂 Processing chb21...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb21_dominant_frequency.csv

📂 Processing chb20...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb20_dominant_frequency.csv

📂 Processing chb19...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb19_dominant_frequency.csv

📂 Processing chb18...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb18_dominant_frequency.csv

📂 Processing chb17...

/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 431, using nperseg = 431
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 207, using nperseg = 207
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 203, using nperseg = 203
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 302, using nperseg = 302
  freqs, _, Pxy 

✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb15_dominant_frequency.csv

📂 Processing chb14...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb14_dominant_frequency.csv

📂 Processing chb13...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb13_dominant_frequency.csv

📂 Processing chb12...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb12_dominant_frequency.csv

📂 Processing chb11...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb11_dominant_frequency.csv

📂 Processing chb10...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb10_dominant_frequency.csv

📂 Processing chb09...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Frequency_Results.final/chb09_dominant_frequency.csv

📂 Processing chb08...
✅ Saved: /Users/puchku-home/KIIT SEM/PROJECT/EEG/Dominant_Freq

/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 454, using nperseg = 454
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 280, using nperseg = 280
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 328, using nperseg = 328
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/puchku-home/KIIT SEM/PROJECT/EEG/myeeg/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 163, using nperseg = 163
  freqs, _, Pxy 